# Day 18: Snailfish
https://adventofcode.com/2021/day/18

### Part 1

In [1]:
import math
import uuid
from dataclasses import dataclass, field
from functools import reduce
from typing import Optional, Union, Tuple, List
from __future__ import annotations
from advent_of_code.utils import input_location, test_input_location

%load_ext blackcellmagic

In [2]:
@dataclass(eq=True, order=True)
class Pair:
    left: Union[Pair, int]
    right: Union[Pair, int]
    uuid: Optional[str] = field(
        default=uuid.uuid1(), repr=False, compare=True
    )  # make unique for comparisons.

    def __add__(self, x: Pair) -> Pair:
        return Pair(self, x)

    def __repr__(self) -> str:
        return f"[{self.left},{self.right}]"

    def explode(self) -> bool:
        """explode"""
        explode_pair = self._find_pair_to_explode()

        # find elements to the left of exploded pair into a flattened list.
        if explode_pair is not None:
            self.left_of(explode_pair)
            self.right_of(explode_pair)
            self.replace_sub_pair(explode_pair, 0)
            return True
        else:
            return False

    def _find_pair_to_explode(self, depth: int = 0) -> Optional[Pair]:
        """Returns left most pair to explode, or None if no pair needs exploding."""
        if isinstance(self.left, int) and isinstance(self.right, int) and depth > 3:
            return self
        else:
            pair = None
            if isinstance(self.left, Pair):
                pair = self.left._find_pair_to_explode(depth=depth + 1)
            if pair is None and isinstance(self.right, Pair):
                pair = self.right._find_pair_to_explode(depth=depth + 1)
            return pair

    def split(self) -> bool:
        """
        Splits left most pair with a number to split, and returns True if
        we split or False if no pair needed splitting.
        """
        if isinstance(self.left, int) and self.left >= 10:
            new_pair = Pair(math.floor(self.left / 2), math.ceil(self.left / 2))
            self.left = new_pair
            return True
        elif isinstance(self.right, int) and self.right > 10:
            new_pair = Pair(math.floor(self.right / 2), math.ceil(self.right / 2))
            self.right = new_pair
            return True
        else:
            success: bool = False
            if isinstance(self.left, Pair):
                success = self.left.split()
            if not success and isinstance(self.right, Pair):
                success = self.right.split()
            return success

    def left_of(self, p: Pair) -> bool:
        if (
            isinstance(self.left, int)
            and isinstance(self.right, Pair)
            and self.right.is_left_most_pair(p)
        ):
            self.left = self.left + p.left
            return True
        elif self.right == p and isinstance(self.left, int):
            self.left = self.left + p.left
            return True
        elif self.right == p and isinstance(self.left, Pair):
            self.left.add_to_right_most_leaf(p.left)
            return True
        else:
            return (self.left.left_of(p) if isinstance(self.left, Pair) else False) or (
                self.right.left_of(p) if isinstance(self.right, Pair) else False
            )

    def right_of(self, p: Pair) -> bool:
        if isinstance(self.right, int) and self.left == p:
            self.right = self.right + p.right
            return True
        elif (
            isinstance(self.right, int)
            and isinstance(self.left, Pair)
            and (self.left.is_right_most_pair(p))
        ):
            self.right = self.right + p.right
            return True
        elif (
            isinstance(self.right, Pair)
            and isinstance(self.left, Pair)
            and (self.left == p or self.left.is_right_most_pair(p))
        ):
            self.right.add_to_left_most_leaf(p.right)
            return True
        else:
            return (
                self.left.right_of(p) if isinstance(self.left, Pair) else False
            ) or (self.right.right_of(p) if isinstance(self.right, Pair) else False)

    def add_to_right_most_leaf(self, x: int):
        """Adds x to right most integer leaf."""
        if isinstance(self.right, int):
            self.right = self.right + x
        else:
            self.right.add_to_right_most_leaf(x)

    def add_to_left_most_leaf(self, x: int):
        """Adds x to left most integer leaf."""
        if isinstance(self.left, int):
            self.left = self.left + x
        else:
            self.left.add_to_left_most_leaf(x)

    def is_left_most_pair(self, p: Pair) -> bool:
        """Is this pair the left most pair of the subtree?"""
        if (
            isinstance(self.left, Pair)
            and isinstance(self.left.left, int)
            and isinstance(self.left.right, int)
            and self.left == p
        ):
            return True
        elif isinstance(self.left, Pair):
            return self.left.is_left_most_pair(p)
        else:
            return False

    def is_right_most_pair(self, p: Pair) -> bool:
        """Is this pair the right most pair of the subtree?"""
        if (
            isinstance(self.right, Pair)
            and isinstance(self.right.left, int)
            and isinstance(self.right.right, int)
            and self.right == p
        ):
            return True
        elif isinstance(self.right, Pair):
            return self.right.is_right_most_pair(p)
        else:
            return False

    def replace_sub_pair(self, p: Pair, value: Union[Pair, int]) -> bool:
        """Replaces the Pair, p, with the value, v. Returns True on success and False on failure"""
        if isinstance(self.left, Pair) and self.left == p:
            self.left = value
            return True
        elif isinstance(self.right, Pair) and self.right == p:
            self.right = value
            return True
        else:
            success = (
                self.left.replace_sub_pair(p, value)
                if isinstance(self.left, Pair)
                else False
            )
            success = (
                success or self.right.replace_sub_pair(p, value)
                if isinstance(self.right, Pair)
                else False
            )
            return success
    
    def reduce(self, print_steps: bool = False):
        """reduces the number one step at a time."""
        while self.explode() or self.split():
            if print_steps:
                print(self)
 
    def magnitude(self) -> int:
        """returns the magnitude of the snailfish number"""
        if isinstance(self.left, int) and isinstance(self.right, int):
            return 3 * self.left + 2 * self.right
        elif isinstance(self.left, int):
            return 3 * self.left + 2 * self.right.magnitude()
        elif isinstance(self.right, int):
            return 3 * self.left.magnitude() + 2 * self.right
        else:
            return 3 * self.left.magnitude() +  2 * self.right.magnitude()


In [3]:
def parse_input(input_file:str) -> List[Pair]:
    """read snailfish number"""
    pairs: list[Pair] = []
    with open(input_file) as f:
        for line in f:
            if line.rstrip():
                snail_fish_number = line.strip()
                snail_fish_number = "subp = "+ snail_fish_number.replace('[', 'Pair(').replace(']', ')')

                # I couldn't be bothered to write a parser for this one... "do as I say, not as I do."
                exec(snail_fish_number, globals())
                pairs.append(globals().get('subp', None))
    return pairs

In [4]:
# Test Explosions
test_1 = Pair(Pair(Pair(Pair(Pair(9,8),1),2),3),4) 
test_1.explode()
assert str(test_1) == "[[[[0,9],2],3],4]"

test_2 = Pair(7,Pair(6,Pair(5,Pair(4,Pair(3,2)))))
test_2.explode()
assert str(test_2) == "[7,[6,[5,[7,0]]]]"

test_3 = Pair(Pair(6,Pair(5,Pair(4,Pair(3,2)))),1)
test_3.explode()
assert str(test_3) == "[[6,[5,[7,0]]],3]"

test_4 = Pair(Pair(3,Pair(2,Pair(1,Pair(7,3)))),Pair(6,Pair(5,Pair(4,Pair(3,2)))))
test_4.explode()
assert str(test_4) == "[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]"
test_4.explode()
assert str(test_4) == "[[3,[2,[8,0]]],[9,[5,[7,0]]]]"

In [5]:
p1_test = Pair(Pair(Pair(Pair(4,3),4),4),Pair(7,Pair(Pair(8,4),9)))
p2_test = Pair(1,1)

# test addition
p3_test = p1_test + p2_test
assert str(p3_test) == "[[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]"

# test 2 explodes
p3_test.explode()
assert str(p3_test) == "[[[[0,7],4],[7,[[8,4],9]]],[1,1]]"

p3_test.explode()
assert str(p3_test) == "[[[[0,7],4],[15,[0,13]]],[1,1]]"

p3_test.split()
assert str(p3_test) == "[[[[0,7],4],[[7,8],[0,13]]],[1,1]]"

p3_test.split()
assert str(p3_test) == "[[[[0,7],4],[[7,8],[0,[6,7]]]],[1,1]]"

p3_test.explode()
assert str(p3_test) == "[[[[0,7],4],[[7,8],[6,0]]],[8,1]]"

In [6]:
p1_test = Pair(Pair(Pair(Pair(4,3),4),4),Pair(7,Pair(Pair(8,4),9)))
p2_test = Pair(1,1)

# test addition
p3_test = p1_test + p2_test
p3_test.reduce()
assert str(p3_test) == "[[[[0,7],4],[[7,8],[6,0]]],[8,1]]"

In [7]:
# unit test magnitude
assert Pair(9,1).magnitude() == 29
assert Pair(1,9).magnitude() == 21
assert Pair(Pair(9,1),Pair(1,9)).magnitude() == 129
assert Pair(Pair(1,2),Pair(Pair(3,4),5)).magnitude() == 143
assert Pair(Pair(Pair(Pair(8,7),Pair(7,7)),Pair(Pair(8,6),Pair(7,7))),Pair(Pair(Pair(0,7),Pair(6,6)),Pair(8,7))).magnitude() == 3488

In [8]:
pairs: list[Pair] = parse_input(test_input_location(day=18))
snailfish_num = reduce(lambda a, b: a + b, pairs)
snailfish_num.reduce()
print(snailfish_num)
snailfish_num.magnitude()  # the answer.

[[[[0,6],[0,6]],[[7,0],[8,0]]],[[[9,7],[0,6]],[[0,10],[9,4]]]]


2608

In [9]:
pairs: list[Pair] = parse_input(input_location(day=18))
snailfish_num = reduce(lambda a, b: a + b, pairs)
snailfish_num.reduce()
print(snailfish_num)
snailfish_num.magnitude()  # the answer.

[[[[1,0],[4,0]],[9,[1,0]]],[[[5,0],6],[[0,1],[0,0]]]]


861

### Part 2